In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [3]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [4]:
from torchvision import datasets, transforms
data_path = 'E:/UNCC/UNCC ML/Project/data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [5]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [6]:
# label_map = {0: 0, 2: 1}
label_map = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}
#class_names = ['airplane', 'bird']
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]

In [23]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 16, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act3 = nn.Tanh()
        self.pool3 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 4 * 4, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 10)
    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = self.pool3(self.act3(self.conv3(out)))
        out = out.view(-1, 8 * 4 * 4)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

In [24]:
model = Net()
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(11138, [864, 32, 4608, 16, 1152, 8, 4096, 32, 320, 10])

In [25]:
img, _ = cifar2[0]
model = Net()
model(img.unsqueeze(0))

tensor([[-0.2428,  0.0497,  0.1319, -0.2047, -0.0246, -0.1069,  0.1678, -0.0412,
          0.1037,  0.1043]], grad_fn=<AddmmBackward0>)

In [26]:
device = (torch.device('cuda') if torch.cuda.is_available()
else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [27]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        epoch_start_time = time.time()
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_train += loss.item()
        epoch_end_time = time.time()   
        if epoch == 1 or epoch % 20 == 0:
            print('{} Time Elapsed {}: Epoch {}, Training loss {}'.format(datetime.datetime.now(),(epoch_end_time - epoch_start_time), epoch, loss_train / len(train_loader)))

In [28]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=32,shuffle=True)

model = Net().to(device=device) #Instantiates our network with GPU ".to(device=device)"
optimizer = optim.SGD(model.parameters(), lr=1e-2) #the stochastic gradient descent optimizer we have been working with
loss_fn = nn.CrossEntropyLoss() #
training_loop(
    n_epochs = 200,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    )

2024-12-09 22:19:50.998674 Time Elapsed 6.993950128555298: Epoch 1, Training loss 2.0181890946508445
2024-12-09 22:21:52.385595 Time Elapsed 6.188495874404907: Epoch 20, Training loss 0.9030771426908953
2024-12-09 22:23:59.827895 Time Elapsed 6.4161646366119385: Epoch 40, Training loss 0.774378925619107
2024-12-09 22:26:07.097126 Time Elapsed 6.115176439285278: Epoch 60, Training loss 0.7164047279574516
2024-12-09 22:28:14.363445 Time Elapsed 6.227900981903076: Epoch 80, Training loss 0.6757261626760851
2024-12-09 22:30:22.970133 Time Elapsed 6.764961242675781: Epoch 100, Training loss 0.6467350502465676
2024-12-09 22:32:32.789033 Time Elapsed 6.410594701766968: Epoch 120, Training loss 0.6246794820384802
2024-12-09 22:34:40.281101 Time Elapsed 6.256637096405029: Epoch 140, Training loss 0.6100863046701033
2024-12-09 22:36:49.150471 Time Elapsed 6.519951581954956: Epoch 160, Training loss 0.5955096396187979
2024-12-09 22:38:55.301026 Time Elapsed 6.325001001358032: Epoch 180, Training 

In [29]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=32, shuffle=False)

val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=32,shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())
        print("Accuracy {}: {:.2f}".format(name , correct / total))
validate(model, train_loader, val_loader)

Accuracy train: 0.80
Accuracy val: 0.67
